# Pulling data from nba_api

Example of data pulled for Kawhi Leonard https://stats.nba.com/player/202695/passes-dash/?Season=2018-19&SeasonType=Playoffs&OpponentTeamID=1610612744&PerMode=Totals

In [1]:
#pip install nba_api

In [2]:
import pandas as pd
from nba_api.stats.static import teams
# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = teams.get_teams()
print('Number of teams fetched: {}'.format(len(nba_teams)))

df = pd.DataFrame.from_dict(nba_teams)
df

Number of teams fetched: 30


,abbreviation,city,full_name,id,nickname,state,year_founded
0,ATL,Atlanta,Atlanta Hawks,1610612737,Hawks,Atlanta,1949
1,BOS,Boston,Boston Celtics,1610612738,Celtics,Massachusetts,1946
2,CLE,Cleveland,Cleveland Cavaliers,1610612739,Cavaliers,Ohio,1970
3,NOP,New Orleans,New Orleans Pelicans,1610612740,Pelicans,Louisiana,2002
4,CHI,Chicago,Chicago Bulls,1610612741,Bulls,Illinois,1966
5,DAL,Dallas,Dallas Mavericks,1610612742,Mavericks,Texas,1980
6,DEN,Denver,Denver Nuggets,1610612743,Nuggets,Colorado,1976
7,GSW,Golden State,Golden State Warriors,1610612744,Warriors,California,1946
8,HOU,Houston,Houston Rockets,1610612745,Rockets,Texas,1967
9,LAC,Los Angeles,Los Angeles Clippers,1610612746,Clippers,California,1970


## Pseudo code

1. Get team IDs for Raptors and Golden State
2. Get all the players on Raptors
    * Might have to get only the Raptors that played in playoffs against GS
3. For each player, get their passes received and passes made
4. Create dataframe with all passes

## Step 1: Get team IDs for Toronto Raptors and Golden State Warriors

In [3]:
team_ids = teams.find_teams_by_full_name('^(Golden State|Toronto)')
team_ids

[{'id': 1610612744,
  'full_name': 'Golden State Warriors',
  'abbreviation': 'GSW',
  'nickname': 'Warriors',
  'city': 'Golden State',
  'state': 'California',
  'year_founded': 1946},
 {'id': 1610612761,
  'full_name': 'Toronto Raptors',
  'abbreviation': 'TOR',
  'nickname': 'Raptors',
  'city': 'Toronto',
  'state': 'Ontario',
  'year_founded': 1995}]

## Step 2: Get all the players on the Raptors in 2018-19 season

In [4]:
# apply custom headers to API request to mimic browser
# otherwise NBA stats times out
custom_headers = {
    'Host': 'stats.nba.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [5]:
# use the CommonTeamRoster endpoint
from nba_api.stats.endpoints import commonteamroster

season='2018-19'
team_id='1610612761'

tor_players = commonteamroster.CommonTeamRoster(season=season, team_id=team_id, headers=custom_headers, timeout=300)
tor_players = tor_players.get_data_frames()[0]

player_ids=tor_players['PLAYER_ID'].tolist()
player_ids

[1627775,
 202695,
 1628384,
 200768,
 1628070,
 201586,
 1626259,
 201980,
 203961,
 202391,
 201975,
 1627832,
 1626181,
 1628449,
 201188,
 1627783]

## Step 3: For each player, get their passes received and passes made

In [16]:
# https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/playerdashptpass.md
from nba_api.stats.endpoints import playerdashptpass

# set up parameters

# required (league_id default is '00' which is NBA)
last_n_games='0'
month='0'
opponent_team_id='1610612744'
per_mode_simple='Totals'
#player_id='1627775'
season='2018-19'
team_id='0'
season_type_all_star='Playoffs'

## Get passes received

In [17]:
passes_received_arr = []

for player_id in player_ids:
    passes_received_temp = playerdashptpass.PlayerDashPtPass(
        last_n_games=last_n_games,
        month=month,
        opponent_team_id=opponent_team_id,
        per_mode_simple=per_mode_simple,
        player_id=str(player_id),
        season=season,
        team_id=team_id,
        season_type_all_star=season_type_all_star,
        headers=custom_headers, timeout=100
    )

    passes_received_df_temp = passes_received_temp.get_data_frames()[1]
    passes_received_arr.append(passes_received_df_temp)

In [18]:
passes_received_arr = pd.concat(passes_received_arr)

In [19]:
passes_received_arr.to_csv('tor_passes_received.csv', index=False)

## Get passes made

In [20]:
passes_made_arr = []

for player_id in player_ids:
    passes_made_temp = playerdashptpass.PlayerDashPtPass(
        last_n_games=last_n_games,
        month=month,
        opponent_team_id=opponent_team_id,
        per_mode_simple=per_mode_simple,
        player_id=str(player_id),
        season=season,
        team_id=team_id,
        season_type_all_star=season_type_all_star,
        headers=custom_headers, timeout=100
    )

    passes_made_df_temp = passes_made_temp.get_data_frames()[0] # yes this could be looped but whatever
    passes_made_arr.append(passes_made_df_temp)

In [21]:
passes_made_arr = pd.concat(passes_made_arr)

In [22]:
passes_made_arr.to_csv('tor_passes_made.csv', index=False)

# Ideas

1. Gantt chart of team founded date https://plot.ly/python/gantt/
2. Something about higher scoring conditional on who passed to whom
3. Shot charts for fun http://savvastjortjoglou.com/nba-shot-sharts.html